In [3]:
firstime = True

In [8]:
folder_path = '/content/drive/My Drive/University/SEM7/CS4642_Data_Mining_and_Information_Retrieval/project_DM_Assignment/'

# Import Statements

In [ ]:
if (firstime):
  from google.colab import drive
  drive.mount('/content/drive')
  !pip install catboost
from pathlib import Path
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score,roc_curve, roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier

import xgboost as xgb
from catboost import CatBoostClassifier

pd.set_option("display.max_columns", 100)

# Load Data

In [10]:
features1_df = pd.read_csv(
    folder_path + "training_set_features.csv", 
    index_col="respondent_id"
)
labels_df = pd.read_csv(
    folder_path + "training_set_labels.csv", 
    index_col="respondent_id"
)
test_features_df = pd.read_csv(folder_path + "test_set_features.csv", 
                               index_col="respondent_id")

In [11]:
frames = [features1_df, test_features_df]
features_df = pd.concat(frames)

# Feature Engineering

In [12]:
features_df['age_group'] = features_df['age_group'].fillna(features_df['age_group'].mode().iloc[0])
features_df['education'] = features_df['education'].fillna(features_df['education'].mode().iloc[0])
features_df['sex'] = features_df['sex'].fillna(features_df['sex'].mode().iloc[0])
features_df['race'] = features_df['race'].fillna(features_df['race'].mode().iloc[0])
features_df['income_poverty'] = features_df['income_poverty'].fillna(features_df['income_poverty'].mode().iloc[0])
features_df['marital_status'] = features_df['marital_status'].fillna(features_df['marital_status'].mode().iloc[0])
features_df['rent_or_own'] = features_df['rent_or_own'].fillna(features_df['rent_or_own'].mode().iloc[0])
features_df['employment_status'] = features_df['employment_status'].fillna(features_df['employment_status'].mode().iloc[0])
features_df['hhs_geo_region'] = features_df['hhs_geo_region'].fillna(features_df['hhs_geo_region'].mode().iloc[0])
features_df['census_msa'] = features_df['census_msa'].fillna(features_df['census_msa'].mode().iloc[0])
features_df['employment_industry'] = features_df['employment_industry'].fillna(features_df['employment_industry'].mode().iloc[0])
features_df['employment_occupation'] = features_df['employment_occupation'].fillna(-1)

In [ ]:
features_df['age_group'] = pd.factorize(features_df.age_group)[0]
features_df['age_group'] = features_df['age_group'].astype("int")
features_df['education'] = pd.factorize(features_df.education)[0]
features_df['education'] = features_df['education'].astype("int")
features_df['sex'] = pd.factorize(features_df.sex)[0]
features_df['sex'] = features_df['sex'].astype("int")
features_df['race'] = pd.factorize(features_df.race)[0]
features_df['race'] = features_df['race'].astype("int")
features_df['income_poverty'] = pd.factorize(features_df.income_poverty)[0]
features_df['income_poverty'] = features_df['income_poverty'].astype("int")
features_df['marital_status'] = pd.factorize(features_df.marital_status)[0]
features_df['marital_status'] = features_df['marital_status'].astype("int")
features_df['rent_or_own'] = pd.factorize(features_df.rent_or_own)[0]
features_df['rent_or_own'] = features_df['rent_or_own'].astype("int")
features_df['employment_status'] = pd.factorize(features_df.employment_status)[0]
features_df['employment_status'] = features_df['employment_status'].astype("int")
features_df['hhs_geo_region'] = pd.factorize(features_df.hhs_geo_region)[0]
features_df['hhs_geo_region'] = features_df['hhs_geo_region'].astype("int")
features_df['census_msa'] = pd.factorize(features_df.census_msa)[0]
features_df['census_msa'] = features_df['census_msa'].astype("int")
features_df['employment_industry'] = pd.factorize(features_df.employment_industry)[0]
features_df['employment_industry'] = features_df['employment_industry'].astype("int")
features_df['employment_occupation'] = pd.factorize(features_df.employment_occupation)[0]
features_df['employment_occupation'] = features_df['employment_occupation'].astype("int")

In [ ]:
features_df['h1n1_concern'] = features_df['h1n1_concern'].fillna(features_df['h1n1_concern'].mode().iloc[0])
features_df['h1n1_knowledge'] = features_df['h1n1_knowledge'].fillna(features_df['h1n1_knowledge'].mode().iloc[0])
features_df['opinion_h1n1_vacc_effective'] = features_df['opinion_h1n1_vacc_effective'].fillna(features_df['opinion_h1n1_vacc_effective'].mode().iloc[0])
features_df['opinion_h1n1_risk'] = features_df['opinion_h1n1_risk'].fillna(features_df['opinion_h1n1_risk'].mode().iloc[0])
features_df['opinion_h1n1_sick_from_vacc'] = features_df['opinion_h1n1_sick_from_vacc'].fillna(features_df['opinion_h1n1_sick_from_vacc'].mode().iloc[0])
features_df['opinion_seas_vacc_effective'] = features_df['opinion_seas_vacc_effective'].fillna(features_df['opinion_seas_vacc_effective'].mode().iloc[0])
features_df['opinion_seas_risk'] = features_df['opinion_seas_risk'].fillna(features_df['opinion_seas_risk'].mode().iloc[0])
features_df['opinion_seas_sick_from_vacc'] = features_df['opinion_seas_sick_from_vacc'].fillna(features_df['opinion_seas_sick_from_vacc'].mode().iloc[0])

In [ ]:
features_df = features_df.fillna(-1)

In [ ]:
features_df = features_df.astype('int')
features_df.shape

In [ ]:
features1_df, test_features_df = features_df[0:26707], features_df[26707:]

In [ ]:
features_df2 = pd.merge(features1_df,labels_df,on ='respondent_id',sort =True)

In [ ]:
features_df2 = features_df2.dropna()

In [ ]:
features_h1n1 = features_df2.drop(columns=['h1n1_vaccine','seasonal_vaccine'])

In [ ]:
features_seasonal = features_df2.drop(columns=['h1n1_vaccine','seasonal_vaccine'])

In [ ]:
y1 = features_df2["h1n1_vaccine"]
y2 = features_df2["seasonal_vaccine"]

In [ ]:
test_features_df_h1n1 = test_features_df

In [ ]:
test_features_df_h1n1

In [ ]:
test_features_df_seasonal = test_features_df

# Model for H1N1

In [ ]:
categorical_var_h1n1 = np.where(features_h1n1.dtypes != np.float)[0]
print('\nCategorical Variables indices : ',categorical_var_h1n1)

In [ ]:
model_h1n1 = CatBoostClassifier(iterations=20000,cat_features=categorical_var_h1n1, nan_mode='Max', depth=8)

In [ ]:
model_h1n1.fit(features_h1n1,y1,plot=False)

# Model for seasonal fever 

In [ ]:
categorical_var_seasonal = np.where(features_seasonal.dtypes != np.float)[0]
print('\nCategorical Variables indices : ',categorical_var_seasonal)

In [ ]:
model_seasonal = CatBoostClassifier(iterations=20000,nan_mode='Max',cat_features=categorical_var_seasonal, depth=8)

In [ ]:
model_seasonal.fit(features_seasonal,y2,plot=False)

# Get Predictions

In [ ]:
predict_h1n1 = model_h1n1.predict_proba(test_features_df_h1n1)
predict_seasonal = model_seasonal.predict_proba(test_features_df_seasonal)

In [ ]:
submission_df = pd.read_csv(folder_path + "submission_format.csv", 
                            index_col="respondent_id")

In [ ]:
# Make sure we have the rows in the same order
np.testing.assert_array_equal(test_features_df.index.values, 
                              submission_df.index.values)

# Save predictions to submission data frame
submission_df["h1n1_vaccine"] = predict_h1n1[:, 1]
submission_df["seasonal_vaccine"] = predict_seasonal[:, 1]

submission_df.head()

In [ ]:
submission_df.to_csv(folder_path + 'Submissions/Mars_rover.csv', index=True)